*This is a Jupyter Notebook. It is an interactive document that contains both rich text elements such as figures, links, equations, etc. and executable code - in this case Python code (the grey boxes).
**How to use a Jupyter Notebook**: You can execute the blocks of code one at the time by placing the mouse in the grey box and pressing shift + enter. An asterisk will appear in the brackets at the top left of the box while the code is being executed (this may take few seconds) and turns into a number when the execution is over. Alternatively, you can run all the code in the Notebook in a single step by clicking on the menu Cell -> Run All.*

# GSA tutorial - What are the key controls of a system?

Francesca Pianosi, Fanny Sarrazin, Andres Penuela-Fernandez

Mathematical models can be used to test hypotheses and increase our understanding of the functioning of the system they represent. Model simulations can be used to explore under which conditions a system will reach a particular state, or to anticipate the effects of human interventions. For example, a mathematical model of an ecological system represents the interactions between different species within a certain habitat. Such a model could be used to identify the conditions under which all the spieces can coexist, or the conditions that would lead to extinction of a particular species; or it could be used to assess the effects of introducing or removing a certain number of individuals of a spieces - for example through hunting or fishing. The underlying assumption here is that the model is a realistic representation of the system, so that what is learnt about the model behaviour applies to the system behaviour too. In this Notebook we show how Global Sensitivity Analysis (GSA) can help in this context, and in particular how GSA can help identify the key controls of the system response.

# An ecological example

For the sake of illustration, we will use a very simple model representing the dynamics going on between two species, of which one (the prey) is the primary source of food source for the other (the predator). The predator-prey system can evolve over time towards reaching a stable equilibrium; or it can create a fluctuating pattern thanks to a negative feedback loop (Ref. 1): when the number of predators grows, the prey population declines, which reduces the predators' food source and soon leads to the predator population to also decline; when predators decline, the pressure on the preys reduces, which enable the prey population to increase again and soon leads to the predator population to also grow. 

<left><img src="eco/predator_prey_equil.gif" width="400px">

Here we will use a simple model to reproduce the predator-prey dynamics (Ref. 2), which uses five parameters:
* **Predator initial population**: the initial number of predator individuals
* **Predator attack rate**: the number of times (per week) that a predator attack is successful, i.e. results in killing and eating a prey
* **Predator efficiency**: the efficiency at turning food into offspring, expressed as the proportion of successful attacks that lead to creating a new predator
* **Predator death rate**: the fraction of the predator population that dies per week
* **Prey initial population**: initial number of prey individuals

The model can be run for a simulation period (for instance 5 years) to reconstruct the temporal evolution of the prey and predator populations over that period. We will define as the **optimal habitat equilibrium** a situation where both the predator and the prey converge over time towards a stable population size (for instance 7000 individuals each). Then we may want to know which of the above parameters mostly control the possibility of reaching such equilibrium or not. This could be useful to inform the sustainable management of the habitat. For instance, if the initial number of preys or predators matter a lot, then a policy of introducing new individuals (preys or predators) would be effective. Or, if predator death rate is influential, then a policy of allowing a certain number of hunting licenses (which is a way to artificially increase the predator death rate) would be effective.

## 1 - Reaching the optimal habitat equilibrium: one-at-the time analysis

In this section, we will run the predator-prey model and see how the simulations change depending on the chosen combination of parameter values. We will vary the parameters one at the time to investigate their impact on the model predictions and whether the populations will follow a fluctuating pattern or stabilise towards an equilibrium value. 

#### Import required libraries

In [2]:
from __future__ import division, absolute_import, print_function
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatRangeSlider, IntRangeSlider
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore') # to hide warning messages
sys.path.append("../../src/")
import safepython.PAWN as PAWN # Module that simulates the ecological model
import safepython.plot_functions as pf # Module to visualize the results
from safepython.model_execution import model_execution # Module to execute the model
from safepython.sampling import AAT_sampling, AAT_sampling_extend # Functions to perform the input sampling
from safepython.util import aggregate_boot # Functions to perform bootstrapping
import eco.predator_prey_model as predator_prey_model # Module that simulates the ecological model


#### Run the model
We are now ready to run the model. Using the slider to change the parameter values, can you find a parameter combination that, over time, lead both the predator and prey population to stabilise? And can you stabilise the populations to a value as close as possible to the optimal habitat equilibrium of 7000 individuals each (blue line)? 

In [3]:
T = 365*5 # days
equil_value = 7 # 7 x 1000 is the equilibrium population
def oat_function(predator_ini = 5, attack_rate = 0.5, efficiency_rate = 0.7, death_rate = 0.05, prey_ini = 10):
    # Set a tentative parameterization:
    param = np.array([predator_ini, attack_rate, efficiency_rate, death_rate, prey_ini])    
    # equil_lim  = 0.5 # ±0.5 x 1000 is the limit to maintain the equilibrium
    # Run simulation:
    predator_pop,prey_pop = predator_prey_model.model(param,T)
    equil_dev = np.array(np.abs(predator_pop[-1]-equil_value)+np.abs(prey_pop[-1]-equil_value))
 
    # Plot results:
    plt.figure(figsize=[15,3])
    plt.plot(prey_pop    , 'm') # prey
    plt.plot(predator_pop, 'c') # predator
    plt.plot(equil_value*np.ones((T,1)),'b' )
    plt.ylabel('individuals (x1000)')
    plt.xlabel('time (days)')
    plt.legend(['predators', 'preys','optimal equilibrium ( = %d' % (equil_value*1000) + " individuals)"  ]) 
    plt.title("Deviation from equilibrium at the end of simulation period = %d" % (equil_dev*1000) + " individuals" )
    plt.ylim((0,20))
    plt.show()
    
interact(oat_function, predator_ini = (2, 10, 1), attack_rate = (0, 1, 0.01), efficiency_rate = (0, 1, 0.01), death_rate = (0, 1, 0.01), prey_ini = (2,10,1));


interactive(children=(IntSlider(value=5, description='predator_ini', max=10, min=2), FloatSlider(value=0.5, de…

## 2 - Finding the key controls of the system response by GSA
Understanding the role of each parameter towards achieving the optimal habitat equilibrium by varying each of them one-at-a-time is not easy. In this section, we will see how Global Sensitivity Analysis (GSA) can be used to analyse the role of parameters in a more structured way and identify the ones that mostly control the long-term system response.

In GSA, we first generate a number of parameter combinations by randomly sampling from the feasible parameter ranges. Then, we run the model against each of these combinations (this is called Monte Carlo simulations). The results of each model run is summarised by a summary scalar output metric. In this case, the output metric is the Deviation from the optimal habitat equilibrium (7000 individuals) at the end of the simulation period:

y = | predators_population(t=end) - 7000 | + | prays_population(t=end) - 7000  |

Last, we will analyse the sample of parameter combinations and associated output metrics through a mix of qualitative (visual analysis) and quantitative (calculating sensitivity indices) approaches.

#### Running Monte Carlo simulations

In [4]:
# Define input variability space:
X_Labels = ['predator_ini', 'attack_rate', 'efficiency_rate', 'death_rate', 'prey_ini'] # Name of parameters (used to customize plots)
M = len(X_Labels) # Number of parameters
distr_fun = st.uniform # Parameter distributions
xmin = [ 2, 0.01, 0.01, 0.01, 2] # Parameter ranges (lower bound)
xmax = [10, 1, 1, 1,10] # Parameter ranges (upper bound)
# Save lower and upper bound in the appropriate format to be passed on to the sampling function:
distr_par = [np.nan] * M
for i in range(M):
    distr_par[i] = [xmin[i], xmax[i] - xmin[i]]
# Choose sampling strategy and size:
samp_strat = 'lhs' # sampling strategy (options: 'lhs' = Latin Hypercube sampling,'rsu' = Random uniform sampling)
# Choose the number of samples:
N = 5000 
# Perform sampling:
X = AAT_sampling(samp_strat, M, distr_fun, distr_par, N)
# Execute the model against all the samples in 'X':
Y = model_execution(predator_prey_model.function, X, T,equil_value)


#### Visual analysis: investigating scatter plots

In [5]:
# Define interactive visualisation function to visualise
# scatter plots of the output metric against parameter samples:
def scatter_function(ymax=100):
    plt.figure(figsize=[15,3])
    for i in range(M):
        plt.subplot(1, M, i+1)
        plt.plot(X[:, i], Y, '.', markerfacecolor='k', markeredgecolor='k')

        # add axis, labels, etc.
        if (i+1) == 1: # first column
            plt.ylabel("y")

        plt.xlabel(X_Labels[i])
        plt.xlim((np.min(X[:, i]), np.max(X[:, i])))
        plt.ylim((0,ymax))
    plt.title("y = Deviation from optimal equilibrium at the end of simulation period (x 1000 individuals)", loc='right')
    plt.show()
interact(scatter_function, ymax = (10, 100, 1));


interactive(children=(IntSlider(value=100, description='ymax', min=10), Output()), _dom_classes=('widget-inter…

#### Questions:

* From these scatter plots, which parameters would you say is most influential in determining the deviation from habitat equilibrium? (it may help to zoom in the lower part of the scatter plots - to do this use the slider to reduce 'ymax')
* Are there parameters that are uninfluential?

#### Calculating sensitivity indices
Next, we formally assess the output sensitivity using one particular GSA method, PAWN (Ref. 3). PAWN provides a sensitivity index for each model parameter. The index measures the relative importance of that parameter on the equilibrium deviation: the lower the index, the smaller the influence. The method also provides a threshold value: if the index of a parameter is above the threshold, then the parameter definitely has an effect on the equilibrium deviation, whereas if the index is below the threshold the effect could be negligible. Now run the cell below to obtain the sensivity indices and the threshold (green line).

<!-- the performance metrics to the model parameters through the PAWN method (Pianosi and Wagener, 2018). Again, the analysis can be repeated twice, once for the RMSE and once for the BIAS. For each performance metric, we can also assess the impact of the choice of the tuning parameters of the PAWN method: the number of conditioning interval (n), the aggregation statistic (median, mean, max) and the number of bootstrap resamples (Nboot) used to estimate confidence intervals of the PAWN indices.-->

In [6]:
# Define interactive visualisation function to calculate and visualise sensitivity indices
# for different choices of the PAWN tuning parameters (n,Nboot,aggr)
def pawn_function(n = 8, aggr = 'median',Nboot = 500):
    # Tuning parameters:
    # n = number of conditioning intervals
    # aggr = statistic to aggregate KS values
    # Nboot = number of bootstrapping resamples used to derive confidence bounds of sensitivity indices   

    # Compute sensitivity indices for Nboot bootstrap resamples
    KS_median, KS_mean, KS_max, KS_dummy = PAWN.pawn_indices(X, Y, n, Nboot=Nboot,dummy=True)
    # KS_median and KS_mean and KS_max have shape (Nboot, M)
        
    # Compute mean and confidence intervals of the sensitivity indices across the bootstrap resamples:
    KS_median_m, KS_median_lb, KS_median_ub = aggregate_boot(KS_median) # shape (M,)
    KS_mean_m, KS_mean_lb, KS_mean_ub = aggregate_boot(KS_mean) # shape (M,)
    KS_max_m, KS_max_lb, KS_max_ub = aggregate_boot(KS_max) # shape (M,)

    # Plot sensitivity indices:
    plt.figure(figsize=[12,5])
    if aggr == 'median':
        pf.boxplot1(KS_median_m, S_lb=KS_median_lb, S_ub=KS_median_ub, X_Labels=X_Labels, Y_Label='sensitivity')
        plt.plot(np.mean(KS_dummy)*np.ones((M+2,1)),'g' )
    if aggr == 'mean':
        pf.boxplot1(KS_mean_m, S_lb=KS_mean_lb, S_ub=KS_mean_ub, X_Labels=X_Labels, Y_Label='sensitivity')
        plt.plot(np.mean(KS_dummy)*np.ones((M+2,1)),'g' )
    if aggr == 'max':
        pf.boxplot1(KS_max_m, S_lb=KS_max_lb, S_ub=KS_max_ub, X_Labels=X_Labels, Y_Label='sensitivity')
        plt.plot(np.mean(KS_dummy)*np.ones((M+2,1)),'g' ) 
    plt.show()
    
interact(pawn_function, n = (5, 10, 1), aggr = ['median', 'mean','max'], Nboot = (0, 500, 10));


interactive(children=(IntSlider(value=8, description='n', max=10, min=5), Dropdown(description='aggr', options…

#### Questions
* Which parameter are most influential? Are these results consistent with the visual inspection of the scatter plots?
* *Advanced GSA question: What is the impact of changing n, Nboot and the aggregation metric? Why?*

#### Visual analysis of interactions
The scatter plots and the sensitivity indices seem to suggest that the attack rate parameter is the most influential in determining deviations from the optimal equilibrium. The second most important parameter is the efficiency rate. The role of the death rate is a bit more unclear; the Predator initial population (predator_ini) and Prey initial population (prey_ini) seem to have little influence. 

In our next step we will then investigate attack_rate, efficiency_rate and death_rate, and look at whether they have significant interactions in determining the deviations from equilibrium. Interactions between pairs of parameters can be visually assessed on the coloured scatter plots below. Each point in these plots represents a parameter combination (you can read the parameter values on the horizontal and vertical axes), coloured according to the value of the equilibrium deviation associated to that combination. Specifically, points highlighted in red represent parameter combinations where the equilibrium deviation is lower than a threshold (y_thres). You can reduce the threshold value to look for combinations that lead the system increasing closer to the optimal equilibrium.

In [8]:
# Define interactive visualisation function to visualise
# scatter plots of input samples against one another:
def scatter_function2(y_thres=5):
    idx = Y[:,0]< y_thres
    plt.figure(figsize=[15,5])
    plt.subplot(131)
    plt.plot(X[:, 1], X[:, 2]   , '.', markerfacecolor='0.8', markeredgecolor='0.8')
    plt.plot(X[idx,1], X[idx, 2], '.', markerfacecolor='r', markeredgecolor='r')
    plt.xlabel(X_Labels[1])
    plt.ylabel(X_Labels[2])
    plt.subplot(132)
    plt.plot(X[:, 1], X[:, 3]   , '.', markerfacecolor='0.8', markeredgecolor='0.8')
    plt.plot(X[idx,1], X[idx, 3], '.', markerfacecolor='r', markeredgecolor='r')
    plt.xlabel(X_Labels[1])
    plt.ylabel(X_Labels[3])
    plt.subplot(133)
    plt.plot(X[:, 2], X[:, 3]   , '.', markerfacecolor='0.8', markeredgecolor='0.8')
    plt.plot(X[idx,2], X[idx, 3], '.', markerfacecolor='r', markeredgecolor='r')
    plt.xlabel(X_Labels[2])
    plt.ylabel(X_Labels[3])
    plt.show()
interact(scatter_function2, y_thres = (1, 10, 1));

interactive(children=(IntSlider(value=5, description='y_thres', max=10, min=1), Output()), _dom_classes=('widg…

#### Interpretation
The first two plots show that there is a very specific sub-range of the Predator attack rate parameter that lead to very low deviations from the optimal equilibrium, whereas outside of this narrow range deviations are above the threshold, regardless of the value that all the other parameters take. The third plot shows that there is a clear interaction between the Predator efficiency ratio and the Predator death rate: low deviations can be achieved for several different values of these parameters but only if they are both increased/reduced simoultaneously.

###  Conclusions: reaching the optimal habitat equilibrium, again...
Now taking into account what we have learnt so far about the key controls of the system response, and about the parameter interactions, can you go back to the first plot (one-at-the-time sensitivity analysis) and find a combination that reach the optimal equilibrium of the predator and prey populations?

## References

1. [Feedback loops: How nature gets its rhythms - Anje-Margriet Neutel](https://ed.ted.com/lessons/feedback-loops-how-nature-gets-its-rhythms-anje-margriet-neutel)
2. [Predator-prey model - Hoppensteadt, F. (2006)](http://www.scholarpedia.org/article/Predator-prey_model)
3. [PAWN method - Pianosi and Wagener (2018)](https://doi.org/10.1016/j.envsoft.2018.07.019)